In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

import seaborn as sns
from collections import defaultdict

In [ ]:
df = pd.read_csv('../data/survey_results_public.csv')
df_schema = pd.read_csv('../data/survey_results_schema.csv')

# Show df to get an idea of the data
df.head()

In [3]:
df_schema.head()

,Column,Question
0,Respondent,Respondent ID number
1,Professional,Which of the following best describes you?
2,ProgramHobby,Do you program as a hobby or contribute to ope...
3,Country,In which country do you currently live?
4,University,"Are you currently enrolled in a formal, degree..."


In [4]:
assert df.shape[1] == df_schema.shape[0]
print('The survey contained {} questions. In total, there are {} survey responses.'.format(df.shape[1], df.shape[0]))

The survey contained 154 questions. In total, there are 19102 survey responses.


### <a class="anchor" id="Exploratory-Data-Analysis">Part I : Which questions should be answered?</a>

The schema and the data are used to find interessting questions which should be answered in the following analysis.

In [1]:
df_schema.head(n=10)

NameError: name 'df_schema' is not defined

**Questions which will be answered in the following parts:**

|#| Question | Additional Information | Helpful columns | Target column |
| ---| :--- | :---| :---| :---|
|4| Does the company influence the happiness/satisfaction of the users?  | only for employed users of a company (EmploymentStatus) | EmploymentStatus, CompanySize, CompanyType, InfluenceInternet, InfluenceWorkstation, InfluenceHardware, InfluenceServers, InfluenceTechStack, InfluenceDeptTech, InfluenceVizTools, InfluenceDatabase, InfluenceCloud, InfluenceConsultants, InfluenceRecruitment, InfluenceCommunication | CareerSatisfaction, JobSatisfaction  |


The questions have been found by looking at the df_schema in detail to find interesting questions.

### <a class="anchor" id="Clean">Part II: Clean the data</a>

In [Part |](#Exploratory-Data-Analysis) the needed columns are defined. In the following, the data is preperaded for each question. Only the needed columns are modified and cleaned.

The column *HaveWorkedLanguage* is split into separated columns, one for each programming language. This information is needed for question 3 and 4, so it is done once at the beginning on the original data.


In [6]:
# HAVE WORKED LANGUAGE - Extract available values
# convert all 'HaveWorkedLanguage' entries into lists: df_q3.HaveWorkedLanguage.str.split(';')
df['HaveWorkedLanguageList'] = df.HaveWorkedLanguage.str.split(';')
#flatten all lists to one
prog_languages = set()
for entry in df.HaveWorkedLanguageList:
    if isinstance(entry, float):
        # list(entry) is empty, so NaN is returned
        continue
    else:
        [prog_languages.add(elem.strip()) for elem in entry]


In [7]:
# convert available programming languages in columns
print(df.columns)
df[list(prog_languages)] = pd.DataFrame(data = np.zeros((df.shape[0],len(prog_languages))), columns = list(prog_languages))
print(df.columns)

Index(['Respondent', 'Professional', 'ProgramHobby', 'Country', 'University',
       'EmploymentStatus', 'FormalEducation', 'MajorUndergrad', 'HomeRemote',
       'CompanySize',
       ...
       'Gender', 'HighestEducationParents', 'Race', 'SurveyLong',
       'QuestionsInteresting', 'QuestionsConfusing', 'InterestedAnswers',
       'Salary', 'ExpectedSalary', 'HaveWorkedLanguageList'],
      dtype='object', length=155)
Index(['Respondent', 'Professional', 'ProgramHobby', 'Country', 'University',
       'EmploymentStatus', 'FormalEducation', 'MajorUndergrad', 'HomeRemote',
       'CompanySize',
       ...
       'VBA', 'Ruby', 'Visual Basic 6', 'Common Lisp', 'Perl', 'R', 'F#',
       'Matlab', 'Haskell', 'VB.NET'],
      dtype='object', length=190)


In [8]:
# Convert HaveWorkedList into binary values for the different programming languages
for idx, prog_lang_list in df.HaveWorkedLanguageList.iteritems():
    if isinstance(prog_lang_list, float):
        # list(entry) is empty, so NaN is returned
        continue
    else:
        for entry in prog_lang_list:
            entry = entry.strip()
            df.loc[df.index[idx], entry] = 1

In [9]:
df.head()

,Respondent,Professional,ProgramHobby,Country,University,EmploymentStatus,FormalEducation,MajorUndergrad,HomeRemote,CompanySize,...,VBA,Ruby,Visual Basic 6,Common Lisp,Perl,R,F#,Matlab,Haskell,VB.NET
0,1,Student,"Yes, both",United States,No,"Not employed, and not looking for work",Secondary school,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Student,"Yes, both",United Kingdom,"Yes, full-time",Employed part-time,Some college/university study without earning ...,Computer science or software engineering,"More than half, but not all, the time",20 to 99 employees,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Professional developer,"Yes, both",United Kingdom,No,Employed full-time,Bachelor's degree,Computer science or software engineering,"Less than half the time, but at least one day ...","10,000 or more employees",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Professional non-developer who sometimes write...,"Yes, both",United States,No,Employed full-time,Doctoral degree,A non-computer-focused engineering discipline,"Less than half the time, but at least one day ...","10,000 or more employees",...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,5,Professional developer,"Yes, I program as a hobby",Switzerland,No,Employed full-time,Master's degree,Computer science or software engineering,Never,10 to 19 employees,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# Pick the needed columns for each question
df_q1 = df[['YearsProgram', 'Overpaid']]
df_q2 = df[list(prog_languages)+['OtherPeoplesCode', 'EnjoyDebugging']]
df_q3 = df[['Python', 'TabsSpaces']]
df_q4 = df[['EmploymentStatus', 'CompanySize', 'CompanyType', 'InfluenceInternet', 'InfluenceWorkstation', 'InfluenceHardware', 
        'InfluenceServers', 'InfluenceTechStack', 'InfluenceDeptTech', 'InfluenceVizTools', 
        'InfluenceDatabase', 'InfluenceCloud', 'InfluenceConsultants', 'InfluenceRecruitment', 
        'InfluenceCommunication', 'CareerSatisfaction', 'JobSatisfaction']]


#### Cleaning for question 4: Does the company influence the happiness/satisfaction of the users?

In [13]:
df_q4.head()

,EmploymentStatus,CompanySize,CompanyType,InfluenceInternet,InfluenceWorkstation,InfluenceHardware,InfluenceServers,InfluenceTechStack,InfluenceDeptTech,InfluenceVizTools,InfluenceDatabase,InfluenceCloud,InfluenceConsultants,InfluenceRecruitment,InfluenceCommunication,CareerSatisfaction,JobSatisfaction
0,"Not employed, and not looking for work",NaN,NaN,Not very satisfied,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Employed part-time,20 to 99 employees,"Privately-held limited company, not in startup...",Satisfied,No influence at all,No influence at all,No influence at all,No influence at all,No influence at all,No influence at all,No influence at all,No influence at all,No influence at all,No influence at all,No influence at all,NaN,NaN
2,Employed full-time,"10,000 or more employees",Publicly-traded corporation,Very satisfied,A lot of influence,Some influence,Some influence,Some influence,A lot of influence,Some influence,Some influence,Some influence,Some influence,Some influence,Some influence,8.0,9.0
3,Employed full-time,"10,000 or more employees",Non-profit/non-governmental organization or pr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,3.0
4,Employed full-time,10 to 19 employees,"Privately-held limited company, not in startup...",Satisfied,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,8.0


In [14]:
# EMPLOYMENT STATUS - Question 1 is only relevant for people who work in a company
print('Answer possibilities for EmploymentStatus: ',df_q4.EmploymentStatus.unique())
df_q4 = df_q4[df_q4.EmploymentStatus.isin(['Employed full-time', 'Employed part-time'])]
df_q4 = df_q4.drop(labels = ['EmploymentStatus'],axis = 1)
print('The number of survey responsed reduced from {} to {}.'.format(df.shape[0], df_q4.shape[0]))

Answer possibilities for EmploymentStatus:  ['Not employed, and not looking for work' 'Employed part-time'
 'Employed full-time'
 'Independent contractor, freelancer, or self-employed'
 'Not employed, but looking for work' 'I prefer not to say' 'Retired']
The number of survey responsed reduced from 19102 to 14823.


In [15]:
# COMPANY SIZE
print('Answer possibilities for CompanySize: ',df_q4.CompanySize.unique())
# remove entries which do not provide additional information
df_q4.CompanySize = df_q4.CompanySize.replace(["I don't know", "I prefer not to answer"], np.NaN)
df_q4 = df_q4.dropna(axis = 0, subset=['CompanySize'])
#print(df_q4.shape)
#df_q4 = pd.concat([df_q4, pd.get_dummies(df_q4.CompanySize, prefix='CompanySize')], ignore_index = True)
tmp = pd.get_dummies(df_q4.CompanySize, prefix='CompanySize')
#print(tmp.shape)
df_q4[tmp.columns] = tmp
#print(df_q4.shape)
df_q4 = df_q4.drop(labels=['CompanySize'], axis = 1)
#print(df_q4.columns)

Answer possibilities for CompanySize:  ['20 to 99 employees' '10,000 or more employees' '10 to 19 employees'
 'Fewer than 10 employees' '5,000 to 9,999 employees'
 '100 to 499 employees' '1,000 to 4,999 employees' '500 to 999 employees'
 "I don't know" nan 'I prefer not to answer']


In [16]:
# COMPANY TYPE
print('Answer possibilities for CompanyTypeSize: ',df_q4.CompanyType.unique())
# remove entries which do not provide additional information
df_q4.CompanyType = df_q4.CompanyType.replace(["I don't know", "I prefer not to answer"], np.NaN)
df_q4 = df_q4.dropna(axis = 0, subset=['CompanyType'])
#print(df_q4.shape)
# df_q4 = pd.concat([df_q4, pd.get_dummies(df_q4.CompanyType, prefix='CompanyType')])
tmp = pd.get_dummies(df_q4.CompanyType, prefix='CompanyType')
#print(tmp.shape)
df_q4[tmp.columns] = tmp
#print(df_q4.shape)
df_q4 = df_q4.drop(labels=['CompanyType'], axis = 1)
#print(df_q4.columns)

Answer possibilities for CompanyTypeSize:  ['Privately-held limited company, not in startup mode'
 'Publicly-traded corporation'
 'Non-profit/non-governmental organization or private school/university'
 'Government agency or public school/university' 'Pre-series A startup'
 'Venture-funded startup' "I don't know"
 'Sole proprietorship or partnership, not in startup mode'
 'I prefer not to answer' 'State-owned company' 'Something else' nan]


In [17]:
# INFLUENCE
influence_columns = ['InfluenceInternet', 'InfluenceWorkstation', 'InfluenceHardware', 
        'InfluenceServers', 'InfluenceTechStack', 'InfluenceDeptTech', 'InfluenceVizTools', 
        'InfluenceDatabase', 'InfluenceCloud', 'InfluenceConsultants', 'InfluenceRecruitment', 
        'InfluenceCommunication']
for column in influence_columns:
    df_q4 = df_q4.dropna(axis = 0, subset=[column])
    tmp = pd.get_dummies(df_q4[column], prefix=column)
    #print(tmp.shape)
    df_q4[tmp.columns] = tmp
    #print(df_q1.shape)
    df_q4 = df_q4.drop(labels=[column], axis = 1)

    
    #df_q1 = df_q1.dropna(axis = 0, subset=[column])
    #df_q1 = pd.concat([df_q1, pd.get_dummies(df_q1[column], prefix=column)])
    #df_q1 = df_q1.drop(labels=[column], axis = 1)
#unique_values


In [18]:
# JOB SATISFACTION
mean = int(df_q4.JobSatisfaction.mean())
# as the satisfaction is on a scale, the mean has been converted to an integer as well
df_q4.JobSatisfaction = df_q4.JobSatisfaction.fillna(mean)

In [19]:
# CAREER SATISFACTION
mean = int(df_q4.CareerSatisfaction.mean())
# as the satisfaction is on a scale, the mean has been converted to an integer as well
df_q4.CareerSatisfaction = df_q4.CareerSatisfaction.fillna(mean)

### <a class="anchor" id="Modelling">Part III: Modelling and Analysis for each question</a>

#### Modelling for question 4: Does the company influence the happiness/satisfaction of the users?
As the data has already be preprocessed, the pipeline only contain the machine learning classifier.

In [30]:
df_q4.head()

,CareerSatisfaction,JobSatisfaction,"CompanySize_1,000 to 4,999 employees",CompanySize_10 to 19 employees,"CompanySize_10,000 or more employees",CompanySize_100 to 499 employees,CompanySize_20 to 99 employees,"CompanySize_5,000 to 9,999 employees",CompanySize_500 to 999 employees,CompanySize_Fewer than 10 employees,...,InfluenceRecruitment_A lot of influence,InfluenceRecruitment_I am the final decision maker,InfluenceRecruitment_No influence at all,InfluenceRecruitment_Not much influence,InfluenceRecruitment_Some influence,InfluenceCommunication_A lot of influence,InfluenceCommunication_I am the final decision maker,InfluenceCommunication_No influence at all,InfluenceCommunication_Not much influence,InfluenceCommunication_Some influence
1,7.0,7.0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,1,0,0
2,8.0,9.0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
6,7.0,6.0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
7,7.0,7.0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,1
8,6.0,6.0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,1,0,0


**Prediction of Career Satisfaction**

In [ ]:
# Logistic Regression with Gridsearch

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV
from sklearn import metrics
from sklearn.ensemble import GradientBoostingRegressor

X = df_q4.drop(labels=['CareerSatisfaction', 'JobSatisfaction'],axis=1)
Y = df_q4.CareerSatisfaction

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3)

params = {
    'n_estimators': [100, 200], 
    'max_depth': [4, 10], 
    'min_samples_split': [2, 5, 10],
    'learning_rate': [0.01, 0.1, 0.5], 
    'loss': ['ls', 'lad', 'huber', 'quantile'] # ls - least squares regression
}

dt =  GradientBoostingRegressor()
lr_gs = GridSearchCV(dt, params, cv=3, verbose=1).fit(X_train, Y_train)

print("Best Params", lr_gs.best_params_)
print("Best Score", lr_gs.best_score_)

#lr_best = LogisticRegression(plug in best params here)
#lr_best.fit(X_train, y_train)
#lr_best.score(X_test, y_test)

Fitting 3 folds for each of 144 candidates, totalling 432 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


**Prediction of JobSatisfaction**

In [33]:
'TODO'

'TODO'